In [63]:
from bs4 import BeautifulSoup
from urllib import request
from time import sleep

IOP_S = "https://iopscience.iop.org/nsearch?terms="
IOP_E = "&nextPage=2&previousPage=-1&currentPage="
IOP_P = "&searchDatePeriod=anytime&orderBy=relevance&pageLength=50"

def souparticlesIOP(material, page, max_int = 7, dream = 2):
    soup1 = None
    ints=0
    while ints<max_int:
        try:
            resp = request.urlopen(IOP_S+material+IOP_E+str(page)+IOP_P)
            soup1 = BeautifulSoup(resp, 'lxml')
            break
        except:
            print("Reintentando pag: " +str(page))
            sleep(dream)
            pass
        finally:
            ints +=1
    if(soup1==None):
        print("Por favor reinicie la busqueda para pag: " + str(page))
        return "Por favor reinicie la busqueda";
    else:
        print("Añadida pag: " + str(page))
        return soup1.find_all('div', {"class" : "art-list-item-body"})

def find_all(material):
    all_     = []
    to_retry = []
    for i in range(1,12):
        item = souparticlesIOP(material, i)
        if(isinstance(item, str)):
            to_retry.append(i)
        else:
            all_.append(item)
    return all_;

In [64]:
a = find_all("ZnTe")

Añadida pag: 1
Reintentando pag: 2
Reintentando pag: 2
Añadida pag: 2
Reintentando pag: 3
Reintentando pag: 3
Añadida pag: 3
Reintentando pag: 4
Reintentando pag: 4
Reintentando pag: 4
Reintentando pag: 4
Reintentando pag: 4
Reintentando pag: 4
Añadida pag: 4
Reintentando pag: 5
Reintentando pag: 5
Añadida pag: 5
Reintentando pag: 6
Reintentando pag: 6
Reintentando pag: 6
Reintentando pag: 6
Reintentando pag: 6
Reintentando pag: 6
Añadida pag: 6
Añadida pag: 7
Reintentando pag: 8
Reintentando pag: 8
Añadida pag: 8
Reintentando pag: 9
Reintentando pag: 9
Reintentando pag: 9
Reintentando pag: 9
Añadida pag: 9
Reintentando pag: 10
Reintentando pag: 10
Añadida pag: 10
Añadida pag: 11


In [66]:

def stringify(raw_pages):
    arts_db = []
    for page in raw_pages:
        for tag in page:
            aa  = i.findChild('div', {"class" : "article-text view-text-small"}).findChild("p")
            bb  = i.findChild('h2',  {"class" : "art-list-item-title"}).findChild("a")
            cc  = i.f('p', {"class" : "small art-list-item-meta"})
    articulos.append([aa.text ,bb.text])
    otras.append(cc)


bs4.element.ResultSet

In [110]:
articulos = []
otras     = []
for i in a[0]:
    title     = i.findChild('h2' , {"class"   : "art-list-item-title"}).findChild("a")          # 0 title
    abstract  = i.findChild('div', {"class"   : "article-text view-text-small"}).findChild("p") # 1 abstract
    DOI       = i.findChild('a'  , {"class"   : "mr-2"})                                        # 2 DOI
    journal   = i.findChild('em')                                                               # 3 Journal
    authors   = i.find_all('span', {"itemprop": "author"})                                      # 4 auths *beta*
    articulos.append([title.text ,abstract.text, DOI.text, journal.text, [auth.getText() for auth in authors] ])


In [111]:
articulos[20][4]

['F Ahmed', 'A En Naciri', 'J J Grob', 'M Stchakovsky', 'L Johann']

In [72]:
otras[5][1]

<p class="small art-list-item-meta">
            2016<em> Mater. Res. Express</em><b> 3</b> 095904
             
            <a class="mr-2" href="https://doi.org/10.1088/2053-1591/3/9/095904">https://doi.org/10.1088/2053-1591/3/9/095904</a>
</p>

In [43]:
articulos[10][0]

'Fe3O4/CdSe/ZnS \nmagnetic fluorescent bifunctional nanocomposites were obtained by depositing heterogeneous\nsemiconductor on magnetic nanoparticles. The structure and properties of the\nFe3O4/CdSe/ZnS \nnanocomposites were fully characterized by TEM, XPS, XRD, SQUID and PL. The results indicate that\nthe Fe3O4/CdSe/ZnS \nnanocomposites are superparamagnetic and are about 8\xa0nm in size.\nThe quantum yield of the nanocomposites increases from 2–3% in\nFe3O4/CdSe to\n10–15% in Fe3O4/CdSe/ZnS, together with a red shift of both the absorption peak and PL band. The increase of\nquantum yield is due to the passivation of the surface of magnetic-CdSe nanocomposites\nwith a ZnS layer. '